In [2]:
!pip install cvxpy

     |████████████████████████████████| 1.3 MB 34.4 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 66.7 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 147 kB 78.5 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 78.5 MB/s eta 0:00:01
     |████████████████████████████████| 941 kB 73.4 MB/s eta 0:00:01
  Created wheel for cvxpy: filename=cvxpy-1.1.15-cp37-cp37m-linux_x86_64.whl size=2620701 sha256=76249493e1f54c29d7f3b9d228baeb1929487270660de0fa9db072d8e6bd6172
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5e/7e/4c/016b4adec886e39f136a72b3c272e4c8c4cbcb3930a83dde0f
  Created wheel for scs: filename=scs-2.1.4-cp37-cp37m-linux_x86_64.whl size=353367 sha256=87f15e8a373e7dfe14ea41abf8dfa94a3b

In [1]:
import time

# start_time = time.time()

# !python optimizer_exp.py

# end_time = time.time()

# print("--{} seconds--".format(end_time - start_time))

In [2]:
import csv
import random
import pandas as pd
import numpy as np

In [ ]:
from Random_Data import RandomPrices

num_assets = 100
days = 100
budget = 1000
problem_number = 5000

price_data = RandomPrices(num_assets, days, budget, problem_number).price_data

with open("Random_Data.csv", "w") as my_csv:
    csvWriter = csv.writer(my_csv, delimiter=',', lineterminator='\n')
    csvWriter.writerows(price_data)

In [3]:
df_price_data = pd.read_csv('Random_Data.csv')
df_price_data.columns = np.arange(1, len(df_price_data.columns) + 1).astype(list)

In [3]:
df_price_data.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,125.746450,226.152916,690.023642,225.341016,791.160680,288.198581,96.023961,387.588160,310.525332,108.226995,...,215.467695,90.335526,1018.178070,590.123906,123.792543,321.249635,549.972013,466.156270,464.238051,268.703704
1,154.747111,265.244938,759.024430,274.393667,1055.033804,329.903568,136.104709,414.905236,232.582924,119.158230,...,217.701905,85.905404,931.372696,488.597849,142.350674,338.980800,501.045854,468.777735,534.411306,264.730056
2,110.948301,245.664568,743.218593,247.264153,700.969188,402.925475,84.790518,575.579213,193.598565,124.299661,...,183.374131,127.883101,810.910025,571.944780,123.477196,361.950391,617.450508,495.955862,498.422340,211.747558
3,159.432248,249.755540,786.294102,191.765749,811.449232,360.040711,100.594633,540.901729,226.467352,127.186097,...,273.329899,91.897258,893.854799,481.320062,115.765952,291.936323,400.436319,616.776735,610.882541,268.070200
4,152.212195,310.674404,887.380249,193.195980,986.147970,377.103815,124.914860,461.259307,266.641338,99.808673,...,267.801305,86.801966,871.749201,526.532794,105.913439,315.674434,565.208811,434.960560,575.639962,249.013912


In [4]:
from PortfolioSelection import PortfolioSelection
from QuboBuilder import QUBO
from MySolveQubo import MySolveQubo

In [5]:
# Choose the starting asset for the sweep
starting_asset = 100
# Choose the highest number of assets being analyzed
highest_assets = 100

theta_one = .3
theta_two = .5
theta_three = .2

slices = 4

# Parameters for anneals
# Choose reverse annealing answer mode. "histogram", 'raw'
f_answer_mode = 'raw'
anneals = 1000
anneal_times = [100]
spin_reversal_dict = {0: '0', 2: '2', 5: '5', 10: '10'}
spin_reversal = 0

In [5]:
for i in range(starting_asset, highest_assets + 1):
    
    problems = 1
    days = 100
    output_array = np.array([])
    output_array_r = np.array([])
    
    for j in range(problems):

        print('Asset #{} for problem #{}'.format(i,j))
        
        # Sets the start and end the the next 100 data points.
        start = j * days
        end = j * days + days

        # slice of price data to use for problem
        price_data = df_price_data[start:end]
        price_data = price_data.values[:, :i]

        # Value for covariance if predetermined, otherwise calculated in Portfolio Selection
        covariance = None

        # ---------------------------------         GET PS TERMS        ---------------------------------------------- #
        # With the PortfolioSelection class import the linear (qi) and quadratic (qij) terms of the QUBO.
        portfolio_selection = PortfolioSelection(theta_one, theta_two, theta_three, price_data,
                                                 slices, covariance)
        qi = portfolio_selection.qi
        qij = portfolio_selection.qij
        # ---------------------------------      GET QUBO FUNCTIONS     ---------------------------------------------- #
        # With the qi and qij terms, form the QUBO matrix and the qubo dictionary.
        qubo = QUBO(qi, qij)
        qubo_matrix = qubo.qubo
        qubo_dict = qubo.qubo_dict

        # initialize the initial state used for reverse annealing
        initial_state = None

        # If it's the first problem of that problem size, embedding = None, else user_embedding is the Embedding found
        # for the j = 0 problem found using the find_embedding() D-Wave function.
        # if j == 0:
        #     user_embedding = None
        # else:
        #     user_embedding = user_embedding
        # Sweep over the anneal times.
#         user_embedding = embedding_group[i * slices]
#         print(user_embedding)
        user_embedding = None
        for k in range(len(anneal_times)):
            # Set the anneal time to be the kth in anneal_times list.
            anneal_time = anneal_times[k]
            # Set the anneal schedule for the forward anneal
            anneal_schedule_f = [[0, 0], [anneal_time, 1]]
            # Solve with forward annealing on the D-Wave_2000Q in a try-try-again loop in case there is a time-out error
            # or something similar.
            result_f = False

            dwave_energy = None
            dwave_array = None
            dwave_num_occurrences = None
            # Place a try-try again piece of code such that if a SAPI error occurs, it runs again.
            result = None


            #########

#             dwave_solve = MySolveQubo(qubo_matrix, qubo_dict, user_embedding, anneals, spin_reversal,
#                                       False, anneal_schedule_f, f_answer_mode, initial_state=None,
#                                       reinitialize=None)
#             dwave_solve.solvequbo()
#             # The dwave energies in order of lowest to highest
#             forward_solution = dwave_solve.dwave_return
#             f_qubo_arrays = dwave_solve.qubo_array

#             sorting = list(dwave_solve.qubo_energy)
#             sorting = np.array([[x, sorting.count(x)] for x in set(sorting)])
#             sorted = sorting[sorting[:, 0].argsort()]
#             num_occ = sorted[:, 1]
#             sorted_energies = sorted[:, 0]
#             sorted_arrays = np.zeros((len(sorted_energies), i * slices))
#             for p in range(len(sorted_energies)):
#                 energy_list = list(dwave_solve.qubo_energy)
#                 index = energy_list.index(sorted_energies[p])
#                 sorted_arrays[p, :] = f_qubo_arrays[index]

#             # The problem ran smoothly on the D-Wave and we got a result.
#             result_f = True

#             np.savez_compressed(
#                 'C:\Users\egy\PycharmProjects\DWave_Solvers\Data\ForwardAnnealing_TestDW5\Embedding_' +
#                 embedding_types[choose_embedding] + '\Slices_' + slice_nums[slices] + '\Anneal_Time' +
#                 anneal_time_dict[
#                     anneal_times[k]] +
#                 '\SpinReversal' + spin_reversal_dict[spin_reversal] + '\Thetas' + thetas[choose_thetas] +
#                 '\Forward' + str(i) + '_' + str(j),
#                 embedding=embedding_group[i * slices],
#                 qubo_dict=qubo_dict,
#                 qubo_matrix=qubo_matrix,
#                 asset_num=i,
#                 problem_num=j,
#                 thetas=thetas[choose_thetas],
#                 spin_reversal=spin_reversal,
#                 anneal_time=anneal_times[k],
#                 arrays=f_qubo_arrays,
#                 embedded_arrays=forward_solution['solutions'],
#                 energies=dwave_solve.qubo_energy,
#                 timing=forward_solution['timing'],
#                 num_occ=num_occ,
#                 sorted_energies=sorted_energies,
#                 sorted_arrays=sorted_arrays,
#                 js=dwave_solve.j0.values(),
#                 hs=dwave_solve.h0,
#                 jc=dwave_solve.jc,
#                 j_scale=dwave_solve.j_scale
#             )

#             ########

#             while not result_f:
#                 try:
#                     dwave_solve = MySolveQubo(qubo_matrix, qubo_dict, user_embedding, anneals, spin_reversal,
#                                               False, anneal_schedule_f, f_answer_mode, initial_state=None,
#                                               reinitialize=None)
#                     dwave_solve.solvequbo()
#                     # The dwave energies in order of lowest to highest
#                     forward_solution = dwave_solve.dwave_return
#                     f_qubo_arrays = dwave_solve.qubo_array

#                     sorting = list(dwave_solve.qubo_energy)
#                     sorting = np.array([[x, sorting.count(x)] for x in set(sorting)])
#                     sorted = sorting[sorting[:, 0].argsort()]
#                     num_occ = sorted[:, 1]
#                     sorted_energies = sorted[:, 0]
#                     sorted_arrays = np.zeros((len(sorted_energies), i * slices))
#                     for p in range(len(sorted_energies)):
#                         energy_list = list(dwave_solve.qubo_energy)
#                         index = energy_list.index(sorted_energies[p])
#                         sorted_arrays[p, :] = f_qubo_arrays[index]

#                     # The problem ran smoothly on the D-Wave and we got a result.
#                     result_f = True

#                     np.savez_compressed(
#                         'C:\Users\egy\PycharmProjects\DWave_Solvers\Data\ForwardAnnealing_TestDW5\Embedding_' +
#                         embedding_types[choose_embedding] + '\Slices_' + slice_nums[slices] + '\Anneal_Time' +
#                         anneal_time_dict[anneal_times[k]] + '\SpinReversal' + spin_reversal_dict[spin_reversal] +
#                         '\Thetas' + thetas[choose_thetas] + '\Forward' + str(i) + '_' + str(j),
#                         embedding=embedding_group[i * slices],
#                         qubo_dict=qubo_dict,
#                         qubo_matrix=qubo_matrix,
#                         asset_num=i,
#                         problem_num=j,
#                         thetas=thetas[choose_thetas],
#                         spin_reversal=spin_reversal,
#                         anneal_time=anneal_times[k],
#                         arrays=f_qubo_arrays,
#                         embedded_arrays=forward_solution['solutions'],
#                         energies=dwave_solve.qubo_energy,
#                         timing=forward_solution['timing'],
#                         num_occ=num_occ,
#                         sorted_energies=sorted_energies,
#                         sorted_arrays=sorted_arrays,
#                         js=dwave_solve.j0.values(),
#                         hs=dwave_solve.h0,
#                         jc=dwave_solve.jc,
#                         j_scale=dwave_solve.j_scale
#                     )

#                 except:
#                     result_f = None
#                     print('ERROR Forward')
#                     pass

Asset #30 for problem #0


NameError: name 'df_price_data' is not defined

In [24]:
qubo_dict

{(0, 0): -0.49998890824777686,
 (0, 1): 0.5000060877969064,
 (0, 2): 0.2500030438984532,
 (0, 3): 0.1250015219492266,
 (0, 4): 1.0000020039014978,
 (0, 5): 0.5000010019507489,
 (0, 6): 0.25000050097537446,
 (0, 7): 0.12500025048768723,
 (0, 8): 0.9999973277101506,
 (0, 9): 0.4999986638550753,
 (0, 10): 0.24999933192753765,
 (0, 11): 0.12499966596376882,
 (0, 12): 1.0000199707173252,
 (0, 13): 0.5000099853586626,
 (0, 14): 0.2500049926793313,
 (0, 15): 0.12500249633966565,
 (0, 16): 0.9999894461404757,
 (0, 17): 0.49999472307023785,
 (0, 18): 0.24999736153511892,
 (0, 19): 0.12499868076755946,
 (0, 20): 1.0000065328384817,
 (0, 21): 0.5000032664192409,
 (0, 22): 0.25000163320962043,
 (0, 23): 0.12500081660481022,
 (0, 24): 0.9999820226965368,
 (0, 25): 0.4999910113482684,
 (0, 26): 0.2499955056741342,
 (0, 27): 0.1249977528370671,
 (0, 28): 0.9999790530298424,
 (0, 29): 0.4999895265149212,
 (0, 30): 0.2499947632574606,
 (0, 31): 0.1249973816287303,
 (0, 32): 0.9999825223772473,
 (0, 33)

In [6]:
my_bucket = f"amazon-braket-1a222675c751" # the name of the bucket
my_prefix = "annealer-experiment" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

In [7]:
import dimod
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
from dimod.binary_quadratic_model import BinaryQuadraticModel
from dwave.system.composites import EmbeddingComposite

In [ ]:
device_arn = 'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6'
device_arn = 'arn:aws:braket:::device/qpu/d-wave/Advantage_system4'

In [ ]:
# testbench for 
starting_asset = 10
highest_assets = 40
starting_slice = 4
highest_slice = 5


for i in range(starting_asset, highest_assets + 1):
    
    problems = 1
    days = 100
    output_array = np.array([])
    output_array_r = np.array([])
    
    j = 0
    
    slice_test = True
    initial_slice_test = True
    for slices in range(starting_slice, highest_slice + 1):
                
        if slice_test == False:
            break

        start_time = time.time()
        
        # Sets the start and end the the next 100 data points.
        start = j * days
        end = j * days + days

        # slice of price data to use for problem
        price_data = df_price_data[start:end]
        price_data = price_data.values[:, :i]

        # Value for covariance if predetermined, otherwise calculated in Portfolio Selection
        covariance = None

        # ---------------------------------         GET PS TERMS        ---------------------------------------------- #
        # With the PortfolioSelection class import the linear (qi) and quadratic (qij) terms of the QUBO.
        portfolio_selection = PortfolioSelection(theta_one, theta_two, theta_three, price_data,
                                                 slices, covariance)
        qi = portfolio_selection.qi
        qij = portfolio_selection.qij
        # ---------------------------------      GET QUBO FUNCTIONS     ---------------------------------------------- #
        # With the qi and qij terms, form the QUBO matrix and the qubo dictionary.
        qubo = QUBO(qi, qij)
        qubo_matrix = qubo.qubo
        qubo_dict = qubo.qubo_dict
        
        end_time = time.time()
        
        initial_time = end_time - start_time
        
        start_time = time.time()
        
        
        try:
            # set parameters
            num_reads = 1
            vartype = dimod.SPIN
            linear = {}
            offset = 0.0
            
            # define BQM
            bqm = dimod.BinaryQuadraticModel(linear, qubo_dict, offset, vartype)
            
            # run BQM: solve with the Advantage_system4 device
            sampler = BraketDWaveSampler(s3_folder,'arn:aws:braket:::device/qpu/d-wave/Advantage_system4')
            
            end_time = time.time()
            
            define_time = end_time - start_time
            
            start_time = time.time()
            sampler = EmbeddingComposite(sampler)
            end_time = time.time()
            
            embed_time = end_time - start_time
            
            start_time = time.time()
            sampleset = sampler.sample(bqm, num_reads=num_reads)
            end_time = time.time()
            
            annealing_time = end_time - start_time
            print('Asset #{} slieces #{} pass'.format(i,slices))
        
        except:
            print("Asset # {} slices # {} fail!!!!!!!!!!!!".format(i, slices))
            slice_test = False
            if slices == starting_slice:
                initial_slice_test = False
            pass
        
        
        print("Asset #{} slieces #{}".format(i, slices))
        
        complete_time = initial_time + define_time + embed_time + annealing_time
        
        print("Complete time: {}s - init time: {}s - define time: {}s - embed time: {}s - anneal time: {}".format(complete_time, initial_time, define_time, embed_time, annealing_time))
        
    if initial_slice_test == False:
        break


Asset #10 slieces #4 pass
Asset #10 slieces #4
Complete time: 14.720426082611084s - init time: 0.03743290901184082s - define time: 2.392361879348755s - embed time: 0.6789190769195557s - anneal time: 11.611712217330933
Asset #10 slieces #5 pass
Asset #10 slieces #5
Complete time: 23.3355495929718s - init time: 0.04824042320251465s - define time: 2.107221841812134s - embed time: 0.5589065551757812s - anneal time: 20.621180772781372
Asset #11 slieces #4 pass
Asset #11 slieces #4
Complete time: 14.355098962783813s - init time: 0.03712129592895508s - define time: 2.1377196311950684s - embed time: 0.6412532329559326s - anneal time: 11.539004802703857
Asset #11 slieces #5 pass
Asset #11 slieces #5
Complete time: 14.446871995925903s - init time: 0.06258511543273926s - define time: 2.0772297382354736s - embed time: 0.6532979011535645s - anneal time: 11.653759241104126
Asset #12 slieces #4 pass
Asset #12 slieces #4
Complete time: 19.2934250831604s - init time: 0.054583072662353516s - define time

In [ ]:
# testbench for 
starting_asset = 10
highest_assets = 40
starting_slice = 4
highest_slice = 5


for i in range(starting_asset, highest_assets + 1):
    
    problems = 1
    days = 100
    output_array = np.array([])
    output_array_r = np.array([])
    
    j = 0
    
    slice_test = True
    initial_slice_test = True
    for slices in range(starting_slice, highest_slice + 1):
                
        if slice_test == False:
            break

        start_time = time.time()
        
        # Sets the start and end the the next 100 data points.
        start = j * days
        end = j * days + days

        # slice of price data to use for problem
        price_data = df_price_data[start:end]
        price_data = price_data.values[:, :i]

        # Value for covariance if predetermined, otherwise calculated in Portfolio Selection
        covariance = None

        # ---------------------------------         GET PS TERMS        ---------------------------------------------- #
        # With the PortfolioSelection class import the linear (qi) and quadratic (qij) terms of the QUBO.
        portfolio_selection = PortfolioSelection(theta_one, theta_two, theta_three, price_data,
                                                 slices, covariance)
        qi = portfolio_selection.qi
        qij = portfolio_selection.qij
        # ---------------------------------      GET QUBO FUNCTIONS     ---------------------------------------------- #
        # With the qi and qij terms, form the QUBO matrix and the qubo dictionary.
        qubo = QUBO(qi, qij)
        qubo_matrix = qubo.qubo
        qubo_dict = qubo.qubo_dict
        
        end_time = time.time()
        
        initial_time = end_time - start_time
        
        start_time = time.time()
        
        
        try:
            # set parameters
            num_reads = 1
            vartype = dimod.SPIN
            linear = {}
            offset = 0.0
            
            # define BQM
            bqm = dimod.BinaryQuadraticModel(linear, qubo_dict, offset, vartype)
            
            # run BQM: solve with the Advantage_system4 device
            sampler = BraketDWaveSampler(s3_folder,'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')
            
            end_time = time.time()
            
            define_time = end_time - start_time
            
            start_time = time.time()
            sampler = EmbeddingComposite(sampler)
            end_time = time.time()
            
            embed_time = end_time - start_time
            
            start_time = time.time()
            sampleset = sampler.sample(bqm, num_reads=num_reads)
            end_time = time.time()
            
            annealing_time = end_time - start_time
            print('Asset #{} slieces #{} pass'.format(i,slices))
        
        except:
            print("Asset # {} slices # {} fail!!!!!!!!!!!!".format(i, slices))
            slice_test = False
            if slices == starting_slice:
                initial_slice_test = False
            pass
        
        
        print("Asset #{} slieces #{}".format(i, slices))
        
        complete_time = initial_time + define_time + embed_time + annealing_time
        
        print("Complete time: {}s - init time: {}s - define time: {}s - embed time: {}s - anneal time: {}".format(complete_time, initial_time, define_time, embed_time, annealing_time))
        
    if initial_slice_test == False:
        break


Asset #10 slieces #4 pass
Asset #10 slieces #4
Complete time: 14.490296840667725s - init time: 0.10871386528015137s - define time: 1.0942773818969727s - embed time: 0.18189382553100586s - anneal time: 13.105411767959595
Asset #10 slieces #5 pass
Asset #10 slieces #5
Complete time: 15.201283931732178s - init time: 0.045912742614746094s - define time: 0.9622414112091064s - embed time: 0.0933082103729248s - anneal time: 14.0998215675354
Asset #11 slieces #4 pass
Asset #11 slieces #4
Complete time: 20.527873277664185s - init time: 0.038855552673339844s - define time: 0.9872610569000244s - embed time: 0.0696110725402832s - anneal time: 19.432145595550537
Asset #11 slieces #5 pass
Asset #11 slieces #5
Complete time: 23.52547287940979s - init time: 0.06036949157714844s - define time: 0.9692044258117676s - embed time: 0.08304166793823242s - anneal time: 22.41285729408264
Asset #12 slieces #4 pass
Asset #12 slieces #4
Complete time: 12.383145093917847s - init time: 0.053618431091308594s - defin

In [11]:
# print solution
print(sampleset)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 ... 19    energy num_oc. ...
0 +1 +1 -1 -1 +1 -1 -1 +1 +1 -1 -1 -1 -1 +1 -1 +1 ... +1 -6.556243       1 ...
['SPIN', 1 rows, 1 samples, 20 variables]


In [15]:
# aggregate solution: 
sampleset = sampleset.aggregate()

In [16]:
print(sampleset)

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 ... 19    energy num_oc. ...
662 +1 -1 -1 -1 +1 -1 -1 -1 -1 +1 +1 -1 -1 +1 +1 ... -1 -9.368885       1 ...
702 +1 +1 -1 +1 -1 -1 -1 +1 -1 +1 -1 -1 -1 +1 +1 ... -1 -9.368884       1 ...
627 -1 -1 -1 +1 +1 +1 -1 -1 +1 -1 -1 +1 +1 -1 +1 ... -1 -9.368882       1 ...
562 -1 -1 -1 +1 -1 +1 +1 +1 +1 +1 -1 +1 +1 -1 +1 ... -1 -9.368882       1 ...
740 -1 -1 -1 +1 +1 +1 +1 -1 +1 +1 -1 -1 +1 -1 -1 ... -1 -9.368881       1 ...
398 -1 -1 +1 +1 +1 +1 +1 -1 +1 -1 -1 +1 +1 -1 -1 ... +1 -9.368879       1 ...
756 -1 -1 +1 -1 +1 +1 -1 -1 +1 +1 -1 +1 +1 -1 -1 ... -1 -9.368873       1 ...
566 +1 -1 +1 +1 -1 -1 +1 -1 +1 -1 +1 -1 -1 +1 +1 ... +1 -9.368863       1 ...
399 +1 +1 -1 +1 -1 -1 -1 -1 -1 +1 +1 -1 -1 +1 +1 ... -1 -9.368861       1 ...
430 +1 -1 -1 -1 +1 -1 -1 +1 -1 +1 +1 -1 -1 +1 +1 ... -1  -9.36886       1 ...
657 +1 -1 -1 -1 +1 +1 +1 -1 -1 -1 +1 +1 -1 +1 -1 ... -1 -9.368855       1 ...
345 -1 -1 +1 +1 +1 +1 -1 +1 +1 -1 -1 +1 +1 -1 -1 ... +1 -9.36883